In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rc
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import re
from wordcloud import WordCloud, STOPWORDS

rc('font', family='Malgun Gothic')
rc('axes', unicode_minus=False)

In [ ]:
import csv
f = open('data_all_0730.csv', 'r', encoding='utf-8')
lines = csv.reader(f)
header = next(lines)
print(header)
f.close()

['brand', 'date', 'rate', 'id', 'skin_type', 'select_1_title', 'select_1_content', 'select_2_title', 'select_2_content', 'select_3_title', 'select_3_content', 'txt']


In [ ]:
# 데이터 불러오기
review = pd.read_csv('data_all_0730.csv', encoding='utf-8')

In [ ]:
# 컬럼 제거
review = review.drop(columns=['date', 'id', 'skin_type','select_1_title','select_2_title','select_3_title'])

In [ ]:
# 컬럼 리네임
review.rename({'brand':'브랜드', 'rate': '평점', 'select_1_content':'피부타입',
               'select_2_content':'피부고민','select_3_content':'자극도', 'txt':'리뷰' }, axis=1, inplace=True)
review.columns

Index(['브랜드', '평점', '피부타입', '피부고민', '자극도', '리뷰'], dtype='object')

In [ ]:
review.head()

,브랜드,평점,피부타입,피부고민,자극도,리뷰
0,닥터지,5점만점에 4점,복합성에 좋아요,진정에 좋아요,보통이에요,컵 받고싶어서 샀어요\n크림은 튜브형태러 내줄 생각은 없는건지 궁금해요
1,닥터지,5점만점에 5점,건성에 좋아요,진정에 좋아요,자극없이 순해요,닥터지는 사계절 내내 잘 사용하고잇어요 ㅎㅎㅎ\n\n물론 건성피부에게 겨울에는 얼굴...
2,닥터지,5점만점에 5점,복합성에 좋아요,진정에 좋아요,보통이에요,아빠 사드렸는데 좋아해요 근데 바르는게 쫌 별로긴 한데 투브타입으로 나오면 더 좋을 듯
3,닥터지,5점만점에 5점,건성에 좋아요,보습에 좋아요,자극없이 순해요,국민 수분 크림이라 그런지 모든 피부타입에도 잘 맞으면서 무난한 아이템이에요 당황이...
4,닥터지,5점만점에 4점,복합성에 좋아요,보습에 좋아요,자극없이 순해요,"- 여름용 수분크림이 필요해서 산 제품이에요.\n- 가격은 세일가로 28,800원 ..."


전처리 작업

In [ ]:
# '\n' 제거
def remove_newline(text):
    return re.sub(r'\n', ' ', text)

review['리뷰'] = review['리뷰'].apply(remove_newline)

In [ ]:
# 평점 중 점수 추출
def score_conversion(text):
    score_match = re.search(r'(\d+)점$', text) # 숫자 다음 '점'이 있는 패턴 찾기, ex) 5점만점에 4점
    if score_match:
        return score_match.group(1) # 추출된 숫자 반환(score_match에서 괄호로 묶인 첫 번째 그룹(숫자) 반환)
    else:
        return None

review['평점'] = review['평점'].apply(score_conversion)

In [ ]:
# 구두점 제거
from string import punctuation

def repreprocessing(text):
    return ''.join(word for word in text if word not in punctuation or word == '+')

review['리뷰'] = review['리뷰'].apply(repreprocessing)

In [ ]:
# 하트 및 '+'를 제외한 구두점 제거
def cleanText(text):
    text = re.sub('[-=,#/\?:;^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》❤️♥️♡]', '', text)
    return text

review['리뷰'] = review['리뷰'].apply(cleanText)

In [ ]:
# 자음 반복 제거
def cleanText2(text):

    text = re.sub('[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]', '', text)
    return text


review['리뷰'] = review['리뷰'].apply(cleanText2)

In [ ]:
# 중복값 확인
review['리뷰'].duplicated().sum()

605

In [ ]:
# 중복 리뷰 제거
review = review.drop_duplicates(subset='리뷰')

In [ ]:
# 중복값 제거 후 확인
review['리뷰'].duplicated().sum()

0

In [ ]:
# 데이터 개수 확인
review.shape

(5495, 6)

In [ ]:
# null 확인
review.isnull().sum()

브랜드     0
평점      0
피부타입    0
피부고민    0
자극도     0
리뷰      0
dtype: int64

In [ ]:
# 전체 평점 중 3점 개수 확인
review['평점'].value_counts()

In [ ]:
# 전체 평점 중 3점 퍼센트 확인
 (review['평점'].value_counts() / len(review)) * 100

In [ ]:
# 3점 리뷰 내용이 애매.... 모델 설계를 위해 빼기로 결정!
review_score3 = review.loc[review['평점'] == 3, '리뷰']
review_score3.head()

In [ ]:
# 전처리 완료 후 모습
review.tail()

,브랜드,평점,피부타입,피부고민,자극도,리뷰
6095,CNP,2,건성에 좋아요,보습에 좋아요,보통이에요,분사구가 너무 별로네요교환했는데도 또 분사구가 한방향으로만 세게 쏘네요재주문 안할 듯
6096,CNP,2,건성에 좋아요,보습에 좋아요,보통이에요,별로물뿌리는거랑차이없어요피부좋아지는건지도모르겠어요
6097,CNP,1,건성에 좋아요,보습에 좋아요,자극이 느껴져요,저는 얼굴이 조금 따가웠어요 양이 많은거 빼고는 비추천
6098,CNP,1,복합성에 좋아요,보습에 좋아요,보통이에요,좋은지별루모르겠어요 좀더사용해야 아직까진딱히모르겠음
6099,CNP,2,건성에 좋아요,보습에 좋아요,보통이에요,향도 좋고 보습도 괜찮은거같네요그런데2개 샀는데샘플은 쪼매난거 하나 주시네요


# 긍정 부정 리뷰 데이터 나누기

In [ ]:
# 긍정 리뷰 데이터
review['평점'] = review['평점'].astype(int)
review_pos = review[review['평점'] >= 4]
review_pos.head()

,브랜드,평점,피부타입,피부고민,자극도,리뷰
0,닥터지,4,복합성에 좋아요,진정에 좋아요,보통이에요,컵 받고싶어서 샀어요 크림은 튜브형태러 내줄 생각은 없는건지 궁금해요
1,닥터지,5,건성에 좋아요,진정에 좋아요,자극없이 순해요,닥터지는 사계절 내내 잘 사용하고잇어요 물론 건성피부에게 겨울에는 얼굴당김잇어서...
2,닥터지,5,복합성에 좋아요,진정에 좋아요,보통이에요,아빠 사드렸는데 좋아해요 근데 바르는게 쫌 별로긴 한데 투브타입으로 나오면 더 좋을 듯
3,닥터지,5,건성에 좋아요,보습에 좋아요,자극없이 순해요,국민 수분 크림이라 그런지 모든 피부타입에도 잘 맞으면서 무난한 아이템이에요 당황이...
4,닥터지,4,복합성에 좋아요,보습에 좋아요,자극없이 순해요,여름용 수분크림이 필요해서 산 제품이에요 가격은 세일가로 28800원 했어요사실...


In [ ]:
review_pos.to_csv('data_all(긍정).csv', index=False)

In [ ]:
# 부정 리뷰 데이터
review['평점'] = review['평점'].astype(int)
review_nag = review[review['평점'] <= 2]
review_nag.head()

,브랜드,평점,피부타입,피부고민,자극도,리뷰
2003,아비브,1,지성에 좋아요,주름/미백에 좋아요,자극이 느껴져요,와 이게 수분크림이 맞나요 바르고 피부 뒤집어져도 나한테만 안맞겠거니 리뷰 안쓰는데...
2028,아비브,1,건성에 좋아요,주름/미백에 좋아요,자극이 느껴져요,트러블 나요 여러번 테스트 해봤는데도 건성 수부지 복합성 이예요 지성인 남자친구도...
2128,아비브,1,지성에 좋아요,진정에 좋아요,자극없이 순해요,수분크림인데 바르고 시간이 조금만 지나도 수분감이 거의 안느껴지고 제형도 별로에요 ...
2161,아비브,2,지성에 좋아요,주름/미백에 좋아요,자극이 느껴져요,지성용 보습제 같습니다 제가 민감건성이긴 한데 수분크림중에 이렇게 안 촉촉한 제품은...
2166,아비브,2,복합성에 좋아요,진정에 좋아요,자극이 느껴져요,생각보다 별로였어요 바르면 얼굴이 붉어지는게 저한테는 안맞는 성분이 있나봅니다 가...


In [ ]:
review_nag.to_csv('data_all(부정).csv', index=False)